In [1]:
pip install -U pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow-gpu==2.8.0

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install protobuf==3.20

Note: you may need to restart the kernel to use updated packages.


# 데이터셋 로드 및 구조 확인

In [6]:
# !wget https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv

--2023-11-29 15:24:36--  https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1319001 (1.3M) [text/plain]
Saving to: ‘finance_data.csv’

finance_data.csv    100%[===================>]   1.26M  --.-KB/s    in 0.05s   

2023-11-29 15:24:36 (27.3 MB/s) - ‘finance_data.csv’ saved [1319001/1319001]



In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('finance_data.csv')

In [9]:
print('샘플의 개수 :', len(df))

샘플의 개수 : 4846


In [10]:
df.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [11]:
# df['labels'] = df['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
# df.head()

,labels,sentence,kor_sentence
0,0,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [12]:
# df.to_csv('finance_data.csv', index=False, encoding='utf-8-sig')

In [13]:
from datasets import load_dataset

In [14]:
all_data = load_dataset(
        "csv",
        data_files={
            "train": "finance_data.csv",
        },
    )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
all_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'sentence', 'kor_sentence'],
        num_rows: 4846
    })
})

In [17]:
cs = all_data['train'].train_test_split(0.2)
train_cs = cs["train"]
test_cs = cs["test"]

In [18]:
train_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3876
})

In [19]:
test_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})

In [20]:
# 훈련 데이터를 다시 8:2로 분리 후 훈련 데이터와 검증 데이터로 저장
cs = train_cs.train_test_split(0.2)
train_cs = cs["train"]
valid_cs = cs["test"]

In [21]:
# 훈련 데이터
train_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3100
})

In [22]:
# 검증 데이터
valid_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 776
})

In [23]:
# 테스트 데이터
test_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})

In [25]:
print('두번째 샘플 출력 :', train_cs['kor_sentence'][1])
print('두번째 샘플의 레이블 출력 :', train_cs['labels'][1])

두번째 샘플 출력 : 최초 예상 총액은 1744900 유로, 최종 수상 금액은 1744900 유로였다.
두번째 샘플의 레이블 출력 : 0


# 데이터셋 전처리

In [26]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm

import csv
import os

import tensorflow as tf
import torch

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [27]:
# 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만듭니다.

train_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', train_cs['kor_sentence']))
validation_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', valid_cs['kor_sentence']))
test_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', test_cs['kor_sentence']))

In [28]:
train_labels = train_cs['labels']
validation_labels = valid_cs['labels']
test_labels = test_cs['labels']

In [29]:
test_sentences[:5]

['[CLS] 고객들은 결제하기 위해 카드를 판독기 앞에서 흔들며 이는 교통 시스템에 사용되는 "터치 앤 고" 카드와 비슷하다. [SEP]',
 '[CLS] 크로스컨트리, 알파인 및 노르딕 워킹 폴, 플로어볼 스틱, 안테나 라돔과 같은 산업용 및 소비재를 위한 고급 복합 제품을 설계, 제조 및 마케팅한다. [SEP]',
 '[CLS] 노르데아 은행(NDA)과 삼포 은행은 핀란드 부동산 투자 회사인 스폰다(SDA1V)가 100만 유로(USD125.4m)의 국내 채권을 발행할 수 있도록 지원했다고 금요일 밝혔다. [SEP]',
 '[CLS] 전 세계 20여 개국에 50개 가까운 서비스 지점 네트워크를 더욱 보완하기 위해 일본에서 더 많은 직원을 모집했다. [SEP]',
 '[CLS] 2009년에 2억 3천 5백만 유로의 순매출액을 기록한 이 그룹은 33개국에서 2,000명 이상의 직원을 고용하고 있다. [SEP]']

In [30]:
test_labels[:5]

[0, 0, 1, 1, 0]

# Electra 토크나이저를 이용한 전처리

In [31]:
from transformers import ElectraTokenizer
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

In [32]:
MAX_LEN = 128

def data_to_tensor (sentences, labels):
  # 정수 인코딩 과정. 각 텍스트를 토큰화한 후에 Vocabulary에 맵핑되는 정수 시퀀스로 변환한다.
  # ex) ['안녕하세요'] ==> ['안', '녕', '하세요'] ==> [231, 52, 45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences는 패딩을 위한 모듈. 주어진 최대 길이를 위해서 뒤에서 0으로 채워준다.
  # ex) [231, 52, 45] ==> [231, 52, 45, 0, 0, 0]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks

In [33]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

In [34]:
print(train_inputs[0])
print(train_masks[0])

tensor([    2,    23,  4217,  4501,  4629,  4234,  6945,  4007,  4417,  4047,
         2969,  4183,  4223,  6312,  7900,  4556,  2024,  4112,  7127,  4073,
         6884,  6887,  6427,    18,    20,  4172,  8533,  4192, 18294,  4172,
         8533,  4239,  6509,  4398,  4176,    18,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [35]:
tokenizer.decode([2])

'[CLS]'

In [36]:
tokenizer.decode([3])

'[SEP]'

In [37]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [38]:
print('훈련 데이터의 크기:', len(train_labels))
print('검증 데이터의 크기:', len(validation_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 3100
검증 데이터의 크기: 776
테스트 데이터의 크기: 970


# GPU가 정상 셋팅되었는지 확인.

In [39]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [40]:
device

device(type='cuda')

# 모델 로드하기

In [41]:
num_labels = 3

In [42]:
from transformers import ElectraForSequenceClassification

model = ElectraForSequenceClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator", num_labels=num_labels)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [43]:
model.cuda()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

# 모델 학습

In [44]:
epochs = 50
batch_size = 16

In [45]:
from torch.nn import functional as F

In [46]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

/home/ubuntu/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [47]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device),
                   attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

 52%|█████▏    | 100/194 [00:30<00:27,  3.44it/s]

Batch Loss: 97.21076613664627 Accuracy: tensor(0.5844, device='cuda:0')


100%|██████████| 194/194 [00:57<00:00,  3.37it/s]


Train Loss: 179.32465201616287 Accuracy: tensor(0.5861, device='cuda:0')


 52%|█████▏    | 100/194 [00:28<00:26,  3.48it/s]

...

Train Loss: 0.5435033631219994 Accuracy: tensor(0.9987, device='cuda:0')


 52%|█████▏    | 100/194 [00:28<00:27,  3.46it/s]

Batch Loss: 0.28852935624308884 Accuracy: tensor(0.9994, device='cuda:0')


100%|██████████| 194/194 [00:55<00:00,  3.47it/s]

Train Loss: 1.7419208855426405 Accuracy: tensor(0.9968, device='cuda:0')


In [48]:
losses, accuracies

([179.32465201616287,
  128.3172515630722,
  82.29570123553276,
  61.313307628035545,
  49.02085709199309,
  35.542063135653734,
  28.0621085036546,
  21.6094886995852,
  16.475483139045537,
  13.979101436212659,
  12.32697840128094,
  12.291142050176859,
  9.12947318283841,
  8.255922310054302,
  7.135359385982156,
  5.277279716916382,
  7.631282634101808,
  5.567261562682688,
  4.7080767878796905,
  3.035715084755793,
  4.633270266233012,
  4.2943902051774785,
  3.6124528656946495,
  3.702212000847794,
  2.9541853590635583,
  3.4659421967808157,
  4.014461063430645,
  2.027977389341686,
  2.8361357641406357,
  2.2109638697002083,
  2.7633123506675474,
  4.5729412866639905,
  2.0424428757978603,
  2.1377356848097406,
  2.603743296524044,
  2.42751361138653,
  3.089555174461566,
  2.0548849033075385,
  1.3449221643386409,
  2.631927368347533,
  1.5937452703947201,
  2.1117993563530035,
  2.923851094674319,
  1.8711956421029754,
  1.4314039726159535,
  1.7055454624351114,
  0.9133487714

# 테스트 데이터에 대한 평가

In [49]:
model.eval()
accum_logits, accum_label_ids = [], []
test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device),
                   attention_mask=attention_masks_batch.to(device)
                  )[0]
    _, predicted = torch.max(y_pred, 1)
    for b in y_pred.cpu().detach().numpy():
        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정
        # ex) [ 3.5134246  -0.30875662 -2.111316  ] ==> 0
        accum_logits.append(np.argmax(b))

    for b in y_batch.cpu().detach().numpy():
        accum_label_ids.append(b)
    
    test_correct += (predicted == y_batch).sum()
    test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

100%|██████████| 61/61 [00:05<00:00, 10.48it/s]

Accuracy: tensor(0.8340, device='cuda:0')


# 메트릭 출력

In [50]:
def metrics(predictions, labels):
    y_pred = predictions
    y_true = labels

    # 사용 가능한 메트릭들을 사용한다.
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)

    # 메트릭 결과에 대해서 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average}

    return metrics

In [51]:
accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

In [52]:
print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))

Accuracy: 0.8340
F1 (Macro) Score: 0.8098
F1 (Micro) Score: 0.8340
F1 (Weighted) Score: 0.8337


In [53]:
# 모델 저장하기
torch.save(model.state_dict(), "model_expert.pt")

# 예측

In [54]:
import numpy as np
from tqdm import tqdm

In [55]:
df = pd.read_csv('./4th_Add Data Colums.csv')

In [56]:
df.head()

,labels,name,theme,동영상 제목,댓글,댓글 작성일,PRO
0,,메이플,BM,"환산10만 루미 장비 및 보스 레이드 [메이플스토리, 뚝이]",환산10만할라면 현질얼마해야되나요,2023-10-07T06:58:56Z,환산10만할라면 현질얼마해야되나요
1,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],11:00 채팅창 보니까 쉘로 넘겼다는 말 하나도 없네;;;,2023-06-20T13:11:19Z,채팅창 보니까 쉘로 넘겼다는 말 하나도 없네
2,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,2023-03-18T01:42:56Z,영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ
3,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],그리고 2년지나고 이재야 생겨난 큐브 천장,2023-04-14T10:01:43Z,그리고 2년지나고 이재야 생겨난 큐브 천장
4,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,2023-03-22T19:55:52Z,근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61220 entries, 0 to 61219
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  50161 non-null  object
 1   name    61220 non-null  object
 2   theme   61220 non-null  object
 3   동영상 제목  61220 non-null  object
 4   댓글      61220 non-null  object
 5   댓글 작성일  61220 non-null  object
 6   PRO     61220 non-null  object
dtypes: object(7)
memory usage: 3.3+ MB


In [58]:
from transformers import pipeline

In [59]:
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512,
                return_all_scores=True, function_to_apply='softmax')

/home/ubuntu/anaconda3/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [60]:
label_dict = {'LABEL_0' : '중립', 'LABEL_1' : '긍정', 'LABEL_2' : '부정'}

In [61]:
# def prediction(text):
#   result = pipe(text)

#   return [label_dict[result[0]['label']]]

In [62]:
def prediction(text):
    results = pipe(text)
    # 가장 높은 점수를 가진 레이블을 찾습니다.
    highest_score_label = max(results[0], key=lambda x: x['score'])['label']
    return label_dict[highest_score_label]

In [63]:
# df["PRO"] 열을 500개씩 나누어 처리
chunks = np.array_split(df['PRO'], np.ceil(len(df) / 500))

# 각 청크에 대해 예측 수행 및 진행 상황을 tqdm으로 표시
predicted_labels = []
for chunk in tqdm(chunks, desc="Predicting"):
    predicted_labels.extend(chunk.apply(prediction))

# 예측된 레이블을 새로운 열에 저장
df['예측 결과_전문가'] = predicted_labels

Predicting:   0%|          | 0/123 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ubuntu/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Predicting: 100%|██████████| 123/123 [09:11<00:00,  4.48s/it]


In [64]:
df['예측 결과_전문가'].value_counts()

중립    54204
부정     4544
긍정     2472
Name: 예측 결과_전문가, dtype: int64

In [65]:
df

,labels,name,theme,동영상 제목,댓글,댓글 작성일,PRO,예측 결과_전문가
0,,메이플,BM,"환산10만 루미 장비 및 보스 레이드 [메이플스토리, 뚝이]",환산10만할라면 현질얼마해야되나요,2023-10-07T06:58:56Z,환산10만할라면 현질얼마해야되나요,중립
1,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],11:00 채팅창 보니까 쉘로 넘겼다는 말 하나도 없네;;;,2023-06-20T13:11:19Z,채팅창 보니까 쉘로 넘겼다는 말 하나도 없네,중립
2,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,2023-03-18T01:42:56Z,영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,중립
3,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],그리고 2년지나고 이재야 생겨난 큐브 천장,2023-04-14T10:01:43Z,그리고 2년지나고 이재야 생겨난 큐브 천장,중립
4,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,2023-03-22T19:55:52Z,근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,중립
...,...,...,...,...,...,...,...,...
61215,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",잼있겠다.. ㅠㅠ 나도 하고싶다,2023-06-30T03:48:27Z,잼있겠다 나도 하고싶다,중립
61216,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",저녀석 잡는거 다시보여주세용ㅋㅋ 오랫만에보니재밋넹,2023-06-01T14:26:42Z,저녀석 잡는거 다시보여주세용ㅋㅋ 오랫만에보니재밋넹,중립
61217,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",저도 요정 하고있는데 스톰샷 마법이 클릭하면 안배워지고 자꾸 땅에 떨어지는데 어떻...,2023-07-08T13:04:39Z,저도 요정 하고있는데 스톰샷 마법이 클릭하면 안배워지고 자꾸 땅에 떨어지는데 어떻...,중립
61218,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",형 가디언 잡는거 보여줘...,2023-07-14T09:14:21Z,형 가디언 잡는거 보여줘,중립


In [66]:
df.to_csv('231201_expert_final.csv', index=False, encoding='utf-8')